# Guidance for training a model with your own data

## 1. Import the necessary packages

In [1]:
import argparse
import os
import random

import numpy as np
import pandas as pd
import torch

from softs.exp.exp_custom import Exp_Custom

## 2. Define the hyperparameters

In [2]:
# fix seed for reproducibility
fix_seed = 7
random.seed(fix_seed)
torch.manual_seed(fix_seed)
np.random.seed(fix_seed)
torch.set_num_threads(6)

# basic config
config = {
    # dataset settings
    'root_path': './dataset/ETT-small/',
    'data_path': 'ETTm1.csv',
    'data': 'ETTm1',
    'features': 'MS',
    'freq': 'B',
    'seq_len': 60,
    'pred_len': 20,
    # model settings
    'model': 'SOFTS',
    'checkpoints': './checkpoints/',
    'd_model': 32,
    'd_core': 16,
    'd_ff': 32,
    'e_layers': 2,
    'learning_rate': 0.0001,
    'lradj': 'cosine',
    'train_epochs': 50,
    'patience': 3,
    'batch_size': 16,
    'dropout': 0.0,
    'activation': 'gelu',
    'use_norm': True,
    'loss_func': 'huber',
    # system settings
    'num_workers': 0,
    'use_gpu': True,
    'gpu': '0',
    'save_model': True,
    'predict_all': True,
}

parser = argparse.ArgumentParser(description='SOFTS')
args = parser.parse_args([])
args.__dict__.update(config)
args.use_gpu = True if torch.cuda.is_available() and args.use_gpu else False

print('Args in experiment:')
print(args)

Args in experiment:
Namespace(root_path='./dataset/ETT-small/', data_path='ETTm1.csv', data='ETTm1', features='MS', freq='B', seq_len=60, pred_len=20, model='SOFTS', checkpoints='./checkpoints/', d_model=32, d_core=16, d_ff=32, e_layers=2, learning_rate=0.0001, lradj='cosine', train_epochs=50, patience=3, batch_size=16, dropout=0.0, activation='gelu', use_norm=True, loss_func='huber', num_workers=0, use_gpu=False, gpu='0', save_model=True, predict_all=True)


In [9]:
from dotenv import load_dotenv
from sqlalchemy import create_engine

load_dotenv()  # take environment variables from .env.

DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_NAME = os.getenv("DB_NAME")
db_url = f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

alchemyEngine = create_engine(
    db_url,
    pool_recycle=3600,
)

In [5]:
conn = alchemyEngine.connect()

In [8]:
cursor = conn.connection.cursor()

In [33]:
# query = """
# with cte as (
# SELECT "date", "open", "close", high, low, volume, amount, open_preclose_rate, high_preclose_rate, low_preclose_rate, vol_change_rate, amt_change_rate, change_rate
# FROM index_daily_em_view 
# where symbol = '399673' 
# order by date desc
# limit 1200
# ) select * from cte order by date
# """
query = """
with cte as (
SELECT "date", "open", "close", high, low, volume, amount, open_preclose_rate, high_preclose_rate, low_preclose_rate, vol_change_rate, amt_change_rate, change_rate
FROM index_daily_em_view 
where symbol = '399673' 
order by date desc

) select * from cte order by date
"""

df = pd.read_sql(query, alchemyEngine, parse_dates=["date"])

In [34]:
df

,date,open,close,high,low,volume,amount,open_preclose_rate,high_preclose_rate,low_preclose_rate,vol_change_rate,amt_change_rate,change_rate
0,2014-06-18,1344.05,1328.84,1351.16,1328.81,3938637.0,8.436718e+09,NaN,NaN,NaN,NaN,NaN,NaN
1,2014-06-19,1327.54,1281.12,1335.44,1269.53,4532795.0,9.384841e+09,-0.09783,0.49667,-4.46329,15.08537,11.23805,-3.59110
2,2014-06-20,1281.14,1294.98,1297.99,1273.18,3111206.0,6.570250e+09,0.00156,1.31682,-0.61977,-31.36231,-29.99082,1.08187
3,2014-06-23,1295.94,1324.14,1330.24,1295.94,3615497.0,7.837101e+09,0.07413,2.72282,0.07413,16.20886,19.28164,2.25177
4,2014-06-24,1322.55,1331.38,1335.14,1320.99,3265263.0,6.941363e+09,-0.12008,0.83073,-0.23789,-9.68702,-11.42946,0.54677
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2440,2024-06-28,1631.33,1616.84,1656.37,1613.74,13468484.0,3.850631e+10,-0.39869,1.13013,-1.47265,35.93660,23.80412,-1.28338
2441,2024-07-01,1612.26,1613.83,1621.57,1581.79,11104284.0,3.785994e+10,-0.28327,0.29255,-2.16781,-17.55357,-1.67860,-0.18617
2442,2024-07-02,1607.48,1600.24,1618.31,1595.66,10769478.0,3.413663e+10,-0.39347,0.27760,-1.12589,-3.01511,-9.83444,-0.84210
2443,2024-07-03,1601.05,1599.10,1614.89,1585.49,10246141.0,3.069951e+10,0.05062,0.91549,-0.92174,-4.85945,-10.06871,-0.07124


In [35]:
df = df.dropna()
df

,date,open,close,high,low,volume,amount,open_preclose_rate,high_preclose_rate,low_preclose_rate,vol_change_rate,amt_change_rate,change_rate
1,2014-06-19,1327.54,1281.12,1335.44,1269.53,4532795.0,9.384841e+09,-0.09783,0.49667,-4.46329,15.08537,11.23805,-3.59110
2,2014-06-20,1281.14,1294.98,1297.99,1273.18,3111206.0,6.570250e+09,0.00156,1.31682,-0.61977,-31.36231,-29.99082,1.08187
3,2014-06-23,1295.94,1324.14,1330.24,1295.94,3615497.0,7.837101e+09,0.07413,2.72282,0.07413,16.20886,19.28164,2.25177
4,2014-06-24,1322.55,1331.38,1335.14,1320.99,3265263.0,6.941363e+09,-0.12008,0.83073,-0.23789,-9.68702,-11.42946,0.54677
5,2014-06-25,1330.12,1322.78,1330.70,1310.50,3214760.0,6.779457e+09,-0.09464,-0.05107,-1.56830,-1.54667,-2.33248,-0.64595
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2440,2024-06-28,1631.33,1616.84,1656.37,1613.74,13468484.0,3.850631e+10,-0.39869,1.13013,-1.47265,35.93660,23.80412,-1.28338
2441,2024-07-01,1612.26,1613.83,1621.57,1581.79,11104284.0,3.785994e+10,-0.28327,0.29255,-2.16781,-17.55357,-1.67860,-0.18617
2442,2024-07-02,1607.48,1600.24,1618.31,1595.66,10769478.0,3.413663e+10,-0.39347,0.27760,-1.12589,-3.01511,-9.83444,-0.84210
2443,2024-07-03,1601.05,1599.10,1614.89,1585.49,10246141.0,3.069951e+10,0.05062,0.91549,-0.92174,-4.85945,-10.06871,-0.07124


In [18]:
df = df[["date", "change_rate"]]
df

,date,change_rate
1,2014-06-19,-3.59110
2,2014-06-20,1.08187
3,2014-06-23,2.25177
4,2014-06-24,0.54677
5,2014-06-25,-0.64595
...,...,...
2440,2024-06-28,-1.28338
2441,2024-07-01,-0.18617
2442,2024-07-02,-0.84210
2443,2024-07-03,-0.07124


In [36]:
df.describe()

,date,open,close,high,low,volume,amount,open_preclose_rate,high_preclose_rate,low_preclose_rate,vol_change_rate,amt_change_rate,change_rate
count,2444,2444.000000,2444.000000,2444.000000,2444.000000,2.444000e+03,2.444000e+03,2444.000000,2444.000000,2444.000000,2444.000000,2444.000000,2444.000000
mean,2019-06-22 17:21:42.127659776,2020.254382,2021.056121,2045.571047,1995.416485,1.078382e+07,3.282664e+10,-0.040256,1.189353,-1.198927,2.278759,2.161632,0.027635
min,2014-06-19 00:00:00,960.230000,970.750000,993.160000,959.500000,2.167637e+06,4.935305e+09,-7.981020,-4.760830,-9.709820,-71.871310,-73.844110,-9.675610
25%,2016-12-15 18:00:00,1547.295000,1548.307500,1563.482500,1529.820000,7.732608e+06,1.517382e+10,-0.312115,0.294597,-1.713448,-12.296585,-12.009170,-1.027045
50%,2019-06-22 12:00:00,1902.600000,1902.455000,1926.830000,1882.795000,1.006884e+07,3.027932e+10,-0.036905,0.866335,-0.893835,-1.121865,-1.606420,-0.033280
75%,2021-12-23 06:00:00,2438.440000,2445.557500,2481.860000,2399.125000,1.298930e+07,4.629271e+10,0.299705,1.758110,-0.332888,12.140453,12.581302,1.104128
max,2024-07-04 00:00:00,3895.610000,3871.400000,3929.050000,3720.170000,3.395136e+07,1.035382e+11,8.111950,8.111950,3.007580,306.093110,297.629870,7.464270
std,NaN,633.413896,632.604339,643.147411,620.810931,4.551554e+06,1.968673e+10,0.851952,1.396905,1.476237,23.121483,22.248692,2.013279


## 3. Prepare the dataset
Organize your data in the following format:
- The dataset should be a csv file.
- If there is a time feature, the first column contains timestamps in the format 'YYYY-MM-DD HH:MM:SS'. If there's no time feature, the dataset starts directly with the features.
- If the parameter `features` is 'M', the following columns are both the features and the targets. If `features` is 'MS', the following columns are the features, and the last column is the target.

In [37]:
# load data
# data = pd.read_csv(os.path.join(args.root_path, args.data_path))
# print(data.head())

# split data
end = int(len(df) * 0.9)
train_data = df.iloc[: end]
vali_data = df.iloc[end - args.seq_len: ]
# test_data = df.iloc[1100 - args.seq_len: ]

# optional: scale data
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
if 'date' in train_data.columns:
    scaler.fit(train_data.iloc[:, 1:])
    train_data.iloc[:, 1:] = scaler.transform(train_data.iloc[:, 1:])
    vali_data.iloc[:, 1:] = scaler.transform(vali_data.iloc[:, 1:])
    # test_data.iloc[:, 1:] = scaler.transform(test_data.iloc[:, 1:])
else:
    scaler.fit(train_data.iloc[:, :])
    train_data.iloc[:, :] = scaler.transform(train_data.iloc[:, :])
    vali_data.iloc[:, :] = scaler.transform(vali_data.iloc[:, :])
    # test_data.iloc[:, :] = scaler.transform(test_data.iloc[:, :])

In [38]:
train_data

,date,open,close,high,low,volume,amount,open_preclose_rate,high_preclose_rate,low_preclose_rate,vol_change_rate,amt_change_rate,change_rate
1,2014-06-19,-1.080209,-1.153426,-1.091164,-1.151741,-1.335967,-1.113222,-0.068981,-0.512218,-2.126917,0.554014,0.402211,-1.762578
2,2014-06-20,-1.150330,-1.132453,-1.146905,-1.146113,-1.637854,-1.251895,0.045675,0.068662,0.396895,-1.457228,-1.436426,0.504097
3,2014-06-23,-1.127964,-1.088328,-1.098904,-1.111023,-1.530763,-1.189478,0.129392,1.064475,0.852539,0.602662,0.760922,1.071569
4,2014-06-24,-1.087750,-1.077372,-1.091610,-1.072402,-1.605139,-1.233611,-0.094649,-0.275616,0.647653,-0.518661,-0.608666,0.244540
5,2014-06-25,-1.076310,-1.090386,-1.098219,-1.088575,-1.615863,-1.241588,-0.065301,-0.900160,-0.225948,-0.166174,-0.202978,-0.334001
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2195,2023-06-26,-3.086317,-3.091908,-3.078724,-3.108894,-2.298546,-1.575607,-1.781333,-1.378456,0.553957,-0.118265,-0.123398,-0.275693
2196,2023-06-27,-3.086303,-3.091903,-3.078743,-3.108878,-2.298546,-1.575607,-0.022833,-1.060692,1.135688,-0.160982,-0.163427,-0.005255
2197,2023-06-28,-3.086312,-3.091921,-3.078777,-3.108916,-2.298546,-1.575607,-0.414207,-1.479023,0.767066,-0.069441,-0.075052,-0.123106
2198,2023-06-29,-3.086318,-3.091935,-3.078763,-3.108886,-2.298546,-1.575607,-0.036527,-1.134061,1.194682,-0.105204,-0.110504,-0.095472


In [39]:
vali_data

,date,open,close,high,low,volume,amount,open_preclose_rate,high_preclose_rate,low_preclose_rate,vol_change_rate,amt_change_rate,change_rate
2140,2023-03-31,-3.085785,-3.091387,-3.078251,-3.108333,-2.298546,-1.575607,0.338930,-1.072094,1.317581,-0.055993,-0.046717,0.035238
2141,2023-04-03,-3.085761,-3.091301,-3.078189,-3.108307,-2.298546,-1.575607,0.568852,-0.609920,1.395486,0.112067,0.028020,0.351201
2142,2023-04-04,-3.085694,-3.091371,-3.078192,-3.108337,-2.298546,-1.575607,0.084922,-1.451298,0.477910,-0.140610,-0.117010,-0.334935
2143,2023-04-06,-3.085805,-3.091375,-3.078260,-3.108358,-2.298546,-1.575607,-0.924900,-1.457212,0.870941,-0.144999,-0.154055,-0.049566
2144,2023-04-07,-3.085779,-3.091327,-3.078195,-3.108316,-2.298546,-1.575607,-0.185829,-0.784057,1.229603,-0.122446,-0.124276,0.183379
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2440,2024-06-28,-0.621113,-0.645411,-0.613488,-0.621052,0.561605,0.321573,-0.416054,-0.063563,-0.163140,1.456897,0.962606,-0.643194
2441,2024-07-01,-0.649932,-0.649965,-0.665285,-0.670311,0.059546,0.289727,-0.282905,-0.656788,-0.619611,-0.859293,-0.173818,-0.110980
2442,2024-07-02,-0.657156,-0.670530,-0.670137,-0.648927,-0.011553,0.106282,-0.410032,-0.667376,0.064556,-0.229759,-0.537534,-0.429146
2443,2024-07-03,-0.666873,-0.672255,-0.675227,-0.664607,-0.122688,-0.063063,0.102271,-0.215584,0.198610,-0.309622,-0.547982,-0.055232


## 4. Train and Evaluate the model


In [40]:
Exp = Exp_Custom(args)
setting = f'{args.data}_{args.model}_{args.seq_len}_{args.pred_len}'
print('>>>>>>>start training : {}>>>>>>>>>>>>>>>>>>>>>>>>>>'.format(setting))
Exp.train(setting=setting, train_data=train_data, vali_data=vali_data, test_data=None)
print('>>>>>>>testing : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
Exp.test(setting=setting, test_data=vali_data)

>>>>>>>start training : ETTm1_SOFTS_60_20>>>>>>>>>>>>>>>>>>>>>>>>>>
>>>>>>>testing : ETTm1_SOFTS_60_20<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<


(0.7294944042827963, 0.5724929982582024, 0.24814378331720302)

## 5. Get predictions by the model

In [24]:
vali_data

,date,change_rate
2140,2023-03-31,0.035238
2141,2023-04-03,0.351201
2142,2023-04-04,-0.334935
2143,2023-04-06,-0.049566
2144,2023-04-07,0.183379
...,...,...
2440,2024-06-28,-0.643194
2441,2024-07-01,-0.110980
2442,2024-07-02,-0.429146
2443,2024-07-03,-0.055232


In [41]:
# get predictions
predictions = Exp.predict(setting=setting, pred_data=vali_data)
print(predictions.shape)

(246, 20, 12)


In [42]:
last_pred2 = predictions[-1, :, :]

In [43]:
it_padded_pred = scaler.inverse_transform(last_pred2)

In [44]:
it_padded_pred

array([[ 1.72926062e+03,  1.72686865e+03,  1.74489355e+03,
         1.71462952e+03,  1.05938800e+07,  3.98001193e+10,
        -5.89669421e-02,  8.62988114e-01, -9.07311678e-01,
         2.63056707e+00,  2.58245730e+00, -1.86177105e-01],
       [ 1.72287305e+03,  1.72105542e+03,  1.73923853e+03,
         1.70885339e+03,  1.04333480e+07,  3.88074783e+10,
        -1.32070065e-01,  8.33381534e-01, -9.16531861e-01,
         1.28995299e+00,  4.22393382e-01, -2.49974847e-01],
       [ 1.72874341e+03,  1.72724268e+03,  1.74481958e+03,
         1.71453772e+03,  1.05205700e+07,  3.92841380e+10,
        -1.84778079e-01,  7.83455849e-01, -9.80892599e-01,
         1.20063233e+00,  5.72999656e-01, -2.28112161e-01],
       [ 1.72846277e+03,  1.72585388e+03,  1.74338196e+03,
         1.71407666e+03,  1.04830710e+07,  3.90761841e+10,
        -1.04344524e-01,  7.14486778e-01, -9.46922243e-01,
         1.34852231e-01, -5.79224765e-01, -2.79890388e-01],
       [ 1.72550977e+03,  1.72416235e+03,  1.7412616

In [29]:
it_padded_pred.shape

(20, 1)

In [45]:
it_padded_pred[:, -1]

array([-0.1861771 , -0.24997485, -0.22811216, -0.2798904 , -0.15068327,
       -0.2187123 , -0.3109654 , -0.22825038,  0.13667627, -0.45120478,
       -0.06703919, -0.27190477, -0.16351332, -0.3224053 , -0.25958896,
       -0.03928372, -0.23652971, -0.26447386, -0.20533572,  0.1468694 ],
      dtype=float32)

In [46]:
it_padded_pred[:, -1].mean()

-0.19252497

In [47]:
it_padded_pred[:, -1].sum()

-3.8504994

In [32]:
train_data.shape

(2199, 2)

In [23]:
# Assuming last_pred is the single column prediction with shape (20, 1)
# and train_data has 13 columns

# Determine the number of columns the scaler was fitted on
num_columns = (
    train_data.shape[1] - 1 if "date" in train_data.columns else train_data.shape[1]
)

# Create an array of zeros with the same number of rows and columns as the scaler's fitted data
padded_pred = np.zeros((last_pred.shape[0], num_columns))

print(padded_pred.shape)

# Insert the single column prediction into the corresponding column (e.g., the first column)
padded_pred[:, 0] = last_pred[:, 0]

# Apply inverse_transform
it_padded_pred = scaler.inverse_transform(padded_pred)

# Extract the inverse transformed single column
it_last_pred = it_padded_pred[:, 0]

print(it_last_pred)

(20, 12)
[3885.40793131 3696.25140262 3774.6123283  3731.39867308 3809.44623464
 3699.65109014 3648.76137504 3721.66227388 3780.73833635 3730.91085114
 3769.92774081 3719.35071022 3732.21961064 3754.28062473 3555.72240312
 3795.2652735  3655.648892   3658.06218308 3602.10752726 3869.83301425]


In [13]:
Exp.metrics

{'epoch': 5,
 'MAE_val': 0.8422023778737977,
 'RMSE_val': 1.1137292115524855,
 'Loss_val': 0.5000086828719738,
 'MAE': 0.3929128,
 'RMSE': 0.5931921,
 'Loss': 0.17228595167398453}